<a href="https://colab.research.google.com/github/RJAbuNasser/Final-Project/blob/main/Final_Project_Girls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Improting Libraries**

In [1]:
import zipfile
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import shapiro

**Core Work**

In [5]:
def plot_data(data):
    data = pd.to_numeric(data, errors='coerce')
    data = data.dropna()
    result = shapiro(data)
    alpha = 0.05
    print("Shapiro-Wilk Normality Check Test:")
    print(f"Statistic: {result.statistic:.3f}, p-value: {result.pvalue:.3f}")
    if result.pvalue > alpha:
        print(f"At {alpha:.3f} significance level: p-value = {result.pvalue:.3f} > {alpha}, data looks normally distributed")
    else:
        print(f"At {alpha:.3f} significance level: p-value = {result.pvalue:.3f} <= {alpha}, data does not look normally distributed")

    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    sns.histplot(data, bins=20, kde=True, ax=axs[0])
    axs[0].set_title('Histogram - KDE')
    axs[0].tick_params(axis='x', labelrotation=45)
    axs[0].tick_params(axis='y', labelrotation=45)

    sns.boxplot(data=data, ax=axs[1])
    axs[1].set_title('Boxplot')
    axs[1].tick_params(axis='x', labelrotation=45)

    sns.violinplot(data=data, ax=axs[2])
    axs[2].set_title('Violin Plot')
    axs[2].tick_params(axis='x', labelrotation=45)
    axs[2].tick_params(axis='y', labelrotation=45)

    plt.subplots_adjust(wspace=0.3, hspace=0.3)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

def enter_txt(file):
    lines = file.readlines()
    data = [line.decode('utf-8').strip().replace(',', ' ').split() for line in lines]
    results = pd.DataFrame(data)
    results.iloc[:, 0] = pd.to_numeric(results.iloc[:, 0], errors='coerce')
    plot_data(results.iloc[:, 0])

def enter_xlsx(file):
    df = pd.read_excel(io.BytesIO(file.read()), sheet_name=None)
    for sheet_name, sheet_data in df.items():
        print(f"Sheet: {sheet_name}")
        sheet_data.iloc[:, 0] = pd.to_numeric(sheet_data.iloc[:, 0], errors='coerce')
        plot_data(sheet_data.iloc[:, 0])

def enter_zip(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for zip_file in zip_ref.namelist():
            if 'Results' in zip_file or '-D' in zip_file:
                folder_name = zip_file.split('/')[0]
                if folder_name.startswith('CEC'):
                    print(f"Entering CEC folder: {folder_name}")
                elif folder_name.startswith('com'):
                    print(f"Entering com folder: {folder_name}")
                elif 'GECCO1' in folder_name:
                    print(f"Entering GECCO1 folder: {folder_name}")
                elif folder_name.startswith('report'):
                    print(f"Entering report folder: {folder_name}")
                if zip_file.endswith('.txt'):
                    print(f"Entering txt file: {zip_file}")
                    with zip_ref.open(zip_file) as file:
                        enter_txt(file)
                elif zip_file.endswith('.xlsx'):
                    print(f"Entering xlsx file: {zip_file}")
                    with zip_ref.open(zip_file) as file:
                        enter_xlsx(file)

enter_zip("Data-All Algos.zip")

FileNotFoundError: [Errno 2] No such file or directory: 'Data-All Algos.zip'

**STD & MErr**

In [ ]:
#till we recieve the actual values from Dr. Mostafa

**Normality Check Tests**

In [4]:
!pip install scikit_posthocs
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, shapiro, wilcoxon, friedmanchisquare
import scikit_posthocs as sp
import zipfile

In [8]:
def plot_data(data):
    data = pd.to_numeric(data, errors='coerce')
    data = data.dropna()
    result = shapiro(data)
    alpha = 0.05
    print("Shapiro-Wilk Normality Check Test:")
    print(f"Statistic: {result.statistic:.3f}, p-value: {result.pvalue:.3f}")
    if result.pvalue > alpha:
        print(f"At {alpha:.3f} significance level: p-value = {result.pvalue:.3f} > {alpha}, data looks normally distributed")
    else:
        print(f"At {alpha:.3f} significance level: p-value = {result.pvalue:.3f} <= {alpha}, data does not look normally distributed")

    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    sns.histplot(data, bins=20, kde=True, ax=axs[0])
    axs[0].set_title('Histogram - KDE')
    axs[0].tick_params(axis='x', labelrotation=45)
    axs[0].tick_params(axis='y', labelrotation=45)

    sns.boxplot(data=data, ax=axs[1])
    axs[1].set_title('Boxplot')
    axs[1].tick_params(axis='x', labelrotation=45)

    sns.violinplot(data=data, ax=axs[2])
    axs[2].set_title('Violin Plot')
    axs[2].tick_params(axis='x', labelrotation=45)
    axs[2].tick_params(axis='y', labelrotation=45)

    plt.subplots_adjust(wspace=0.3, hspace=0.3)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()
def ttest(data, reference):
    results = {}
    for group, values in data.items():
        if group != reference and len(data[reference]) == len(values):
            stat, p_value = ttest_ind(data[reference], values)
            results[group] = p_value
    return results
def perform_wilcoxon(data, reference):
    results = {}
    for group, values in data.items():
        if group != reference and len(data[reference]) == len(values):
            stat, p_value = wilcoxon(data[reference], values)
            results[group] = p_value
    return results
def perform_friedman(data):
    values = [data[group] for group in data]
    stat, p_value = friedmanchisquare(*values)
    return stat, p_value
def post_hoc_analysis(data, alpha=0.05):
    groups = list(data.keys())
    data_array = np.array([data[group] for group in groups]).T
    bonferroni = sp.posthoc_dunn(data_array, p_adjust='bonferroni')
    holm = sp.posthoc_dunn(data_array, p_adjust='holm')
    holland = sp.posthoc_dunn(data_array, p_adjust='holland')
    hochberg = sp.posthoc_dunn(data_array, p_adjust='hochberg')
    hommel = sp.posthoc_dunn(data_array, p_adjust='hommel')
    return {
        'Bonferroni': bonferroni,
        'Holm': holm,
        'Holland': holland,
        'Hochberg': hochberg,
        'Hommel': hommel
    }

####################################################################

def process_txt(file, actual_file, algorithm_folder):
    data = np.loadtxt(file, delimiter=' ')
    actual = np.loadtxt(actual_file, delimiter=' ')
    merrors = data - actual
    return {algorithm_folder: merrors.tolist()}
def process_xlsx(file, actual_file, algorithm_folder):
    df = pd.read_excel(file, sheet_name=None)
    actual = pd.read_excel(actual_file, sheet_name=None)
    merrors = {}
    for sheet_name, sheet_df in df.items():
        actual_sheet = actual[sheet_name]
        merrors[sheet_name] = (sheet_df.iloc[:, 1] - actual_sheet.iloc[:, 1]).tolist()
    return {f"{algorithm_folder}_{sheet_name}": value for sheet_name, value in merrors.items()}
def analyze_grouped_data(data, reference_group, output_file_prefix, use_summary_files=False):
    if use_summary_files:
        summary_file = f"{output_file_prefix}_Summary.xlsx"
        summary_df = pd.read_excel(summary_file)
        summarized_data = {row['Group']: [row['Mean Error']] for index, row in summary_df.iterrows()}

    else:
        summarized_data = data

    summary = []
    for group, values in summarized_data.items():
        summary.append({
            "Group": group,
            "Mean Error": np.mean(values),
            "Std Dev": np.std(values),
            "Normality": test_normality({group: values})[group]
        })

    friedman_stat, friedman_p = perform_friedman(summarized_data)
    wilcoxon_results = perform_wilcoxon(summarized_data, reference_group)
    post_hoc_results = post_hoc_analysis(summarized_data)

    summary_df = pd.DataFrame(summary)
    summary_df.to_excel(f"{output_file_prefix}_Summary.xlsx", index=False)

    for method, results_df in post_hoc_results.items():
        results_df.to_excel(f"{output_file_prefix}_PostHoc_{method}.xlsx", index=True)
def process_zip(data_zip_path, actual_values_zip_path, use_summary_files=False):
    with zipfile.ZipFile(data_zip_path, 'r') as data_zip, zipfile.ZipFile(actual_values_zip_path, 'r') as actual_zip:
        for folder in ["10-D", "20-D"]:
            files_by_folder = {file.split('/')[0]: [] for file in data_zip.namelist() if folder in file}
            for file in data_zip.namelist():
                if folder in file:
                    files_by_folder[file.split('/')[0]].append(file)
            for algorithm_folder, files in files_by_folder.items():
                reference_group = None
                for i, file in enumerate(files):
                    if file in data_zip.namelist() and file in actual_zip.namelist():
                        folder_name = file.split('/')[0]
                        if file.endswith('.txt'):
                            with data_zip.open(file) as f, actual_zip.open(file) as af:
                                data = process_txt(f, af, f"{algorithm_folder}_{folder}_{i}")
                        elif file.endswith('.xlsx'):
                            with data_zip.open(file) as f, actual_zip.open(file) as af:
                                data = process_xlsx(f, af, f"{algorithm_folder}_{folder}_{i}")

                        if i == 0:
                            reference_group = list(data.keys())[0]

                        output_file_prefix = f"{algorithm_folder}_{folder}_Analysis_{i}"
                        analyze_grouped_data(data, reference_group, output_file_prefix, use_summary_files)
process_zip("Data-All Algos.zip", "Actual Values.zip", use_summary_files=True)
